Traning, and Experiments without Feature Engineering

In [28]:
# First experiment without including feature engineering

import pandas as pd
df_preprocessed = pd.read_csv('Datasets/ds_preprocessed.csv')
pd.set_option('display.max_columns', None)
df_preprocessed.head()

,age,minutes_played,points,rebounds,assists,steals,blocks,turnovers,fg_pct,three_pct,ft_pct,plus_minus,efficiency,rest_days,target,position_PF,position_PG,position_SF,position_SG,team_BKN,team_BOS,team_CHA,team_CHI,team_CLE,team_DAL,team_DEN,team_DET,team_GSW,team_HOU,team_IND,team_LAC,team_LAL,team_MEM,team_MIA,team_MIL,team_MIN,team_NOP,team_NYK,team_OKC,team_ORL,team_PHI,team_PHX,team_POR,team_SAC,team_SAS,team_TOR,team_UTA,team_WAS,opponent_BKN,opponent_BOS,opponent_CHA,opponent_CHI,opponent_CLE,opponent_DAL,opponent_DEN,opponent_DET,opponent_GSW,opponent_HOU,opponent_IND,opponent_LAC,opponent_LAL,opponent_MEM,opponent_MIA,opponent_MIL,opponent_MIN,opponent_NOP,opponent_NYK,opponent_OKC,opponent_ORL,opponent_PHI,opponent_PHX,opponent_POR,opponent_SAC,opponent_SAS,opponent_TOR,opponent_UTA,opponent_WAS,game_location_Home
0,25,12.2,9,3,0,1,1,2,51.4,19.6,79.9,-3,12,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,25,30.9,15,14,5,1,2,0,41.0,31.2,68.1,1,37,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,30,18.9,11,4,2,1,0,2,47.4,28.0,78.2,-2,16,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,27,20.0,12,4,2,0,0,3,49.3,45.8,68.1,7,15,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,29,27.8,16,10,4,1,0,2,34.5,22.5,76.0,7,29,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [3]:
# Test split for the experiment without feature engineering
from sklearn.model_selection import train_test_split

# Defining features and target
X_1 = df_preprocessed.drop(columns='target')
y_1 = df_preprocessed['target']

# Test Split (20% of total data)
X_1_train_valid, X_1_test, y_1_train_valid, y_1_test = train_test_split(
    X_1, y_1, test_size=0.2, random_state=12345
)

# Validation split (25% of the remaining 80% → 20% of total)
X_1_train, X_1_valid, y_1_train, y_1_valid = train_test_split(
    X_1_train_valid, y_1_train_valid, test_size=0.25, random_state=12345
)

print(
    f"Training set Rows: {X_1_train.shape[0]} - ",
    f"Validation set Rows: {X_1_valid.shape[0]} - ",
    f"Test set Rows: {X_1_test.shape[0]}"
)

Training set Rows: 5400 -  Validation set Rows: 1800 -  Test set Rows: 1800


Checking Class Balance in the Training Dataset:

In [4]:
# Assuming y_train is a Pandas Series
# The primary reason for checking the class balance is to prevent the model from being biased toward the majority class.
class_counts = y_1_train.value_counts()
print("class counts")
print(class_counts)

class_proportions = y_1_train.value_counts(normalize=True)
print("class Proportions")
print(class_proportions)

class counts
target
0    3212
1    2188
Name: count, dtype: int64
class Proportions
target
0    0.594815
1    0.405185
Name: proportion, dtype: float64


Class Balance Conclusion:
- The dataset has a ~60/40 split between the two classes.
- Is this imbalanced? Yes, it is imbalanced because the classes are not equal (50/50).
- Is it severely imbalanced? No. A severe imbalance is usually considered to be anything worse than 80/20 or 90/10.

Implications:
- It is manageable, but it still requires attention when predicting the minority class.
- Better to use the Precision, Recall, F1-Score, and ROC AUC. To check how well perform with the minority class.
- Adjust class weighting: for XGB set the "scale_pos_weight" parameter. Starting value is the ratio of negative to positive samples 3212/2188 = 1.47. It tells the model to give the monority class 1.47 times the importance of the majority class.

First Model Training with Initial Parameters (no Hyperopt) and without Feature Engineering

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix

CLASS_WEIGHT_RATIO = 3212 / 2188 # 0 class / 1 class

# Initial XGBoost Classification Parameters
initial_clf_params = {
    'n_estimators': 200,          # Number of trees
    'max_depth': 5,               # Maximum depth of each tree
    'learning_rate': 0.1,         # Boosting learning rate
    'subsample': 0.8,             # % of rows sampled per tree
    'colsample_bytree': 0.8,      # % of features sampled per tree
    'objective': 'binary:logistic', # binary classification funtion loss
    'eval_metric': 'logloss',     # binary cross-entropy loss
    'random_state': 12345,        # Reproducibility
    'tree_method': 'hist',         # Histogram-based Tree Construction Algorithm: Fast + recommended (Perfect for Sagemaker)
    'scale_pos_weight': CLASS_WEIGHT_RATIO     # Adjust Class Weighting
}


# Initialize XGBoost Classifier
xgb_clf = XGBClassifier(**initial_clf_params)

# Train the model
xgb_clf.fit(
    X_1_train, y_1_train,
    eval_set=[(X_1_valid, y_1_valid)],
    verbose=False
)

# -------------------------
# VALIDATION PREDICTIONS
# -------------------------
y_val_pred = xgb_clf.predict(X_1_valid) # compute metrics that need discrete labels: accuracy, precision, recall, F1-score, confusion matrix, etc (Ratios)
y_val_proba = xgb_clf.predict_proba(X_1_valid)[:, 1] # probability of the positive class (used for ROC-AUC and any thresholding decisions)

val_acc  = accuracy_score(y_1_valid, y_val_pred)  # Fraction of correct predictions
val_roc  = roc_auc_score(y_1_valid, y_val_proba)  # Measures the model’s ability to distinguish between classes
val_prec = precision_score(y_1_valid, y_val_pred) # Fraction of true positives among predicted positives
val_rec  = recall_score(y_1_valid, y_val_pred)    # Fraction of true positives among actual positives
val_f1   = f1_score(y_1_valid, y_val_pred)        # Harmonic mean of precision and recall

print("\n📊 Validation Results:")
print(f"Accuracy: {val_acc:.4f}")
print(f"ROC AUC: {val_roc:.4f}")
print(f"Precision: {val_prec:.4f}")
print(f"Recall (Sensitivity): {val_rec:.4f}")
print(f"F1-Score: {val_f1:.4f}")

# Confusion Matrix (Validation)
val_cm_1 = confusion_matrix(y_1_valid, y_val_pred)
print("\n🔷 Validation Confusion Matrix:")
print("[[True_Pos False_Pos]") 
print("[False_Neg True_Neg]]")
print(val_cm_1)

# --------------------
# TEST PREDICTIONS
# --------------------
y_test_pred = xgb_clf.predict(X_1_test) # compute metrics that need discrete labels: accuracy, precision, recall, F1-score, confusion matrix, etc (Ratios)
y_test_proba = xgb_clf.predict_proba(X_1_test)[:, 1] # probability of the positive class (used for ROC-AUC and any thresholding decisions)

test_acc  = accuracy_score(y_1_test, y_test_pred)  # Fraction of correct predictions
test_roc  = roc_auc_score(y_1_test, y_test_proba)  # Measures the model’s ability to distinguish between classes
test_prec = precision_score(y_1_test, y_test_pred) # Fraction of true positives among predicted positives
test_rec  = recall_score(y_1_test, y_test_pred)    # Fraction of true positives among actual positives
test_f1   = f1_score(y_1_test, y_test_pred)        # Harmonic mean of precision and recall   

print("\n----------------------------------------")
print("📊 Test Results:")
print(f"Accuracy: {test_acc:.4f}")
print(f"ROC AUC: {test_roc:.4f}")
print(f"Precision: {test_prec:.4f}")
print(f"Recall (Sensitivity): {test_rec:.4f}")
print(f"F1-Score: {test_f1:.4f}")

# Confusion Matrix (Test)
test_cm_1 = confusion_matrix(y_1_test, y_test_pred)
print("\n🔷 Test Confusion Matrix:")
print("[[True_Pos False_Pos]")
print("[False_Neg True_Neg]]")
print(test_cm_1)


📊 Validation Results:
Accuracy: 0.8444
ROC AUC: 0.9243
Precision: 0.7926
Recall (Sensitivity): 0.8299
F1-Score: 0.8108

🔷 Validation Confusion Matrix:
[[True_Pos False_Pos]
[False_Neg True_Neg]]
[[920 157]
 [123 600]]

----------------------------------------
📊 Test Results:
Accuracy: 0.8367
ROC AUC: 0.9230
Precision: 0.7786
Recall (Sensitivity): 0.8012
F1-Score: 0.7897

🔷 Test Confusion Matrix:
[[True_Pos False_Pos]
[False_Neg True_Neg]]
[[954 157]
 [137 552]]


MLflow Experiment Creation

In [4]:
# --- Create MLflow Experiment ---
import mlflow
from mlflow import MlflowClient

client = MlflowClient(tracking_uri="http://127.0.0.1:5000/")

# Set up MLflow experiment
experiment_description = (
    "This is the WZ challenge about NBA player performance"
    "This experiment contains an XGBoost model with hyperparameter tuning using Hyperopt."
)
experiment_tags = {
    "team": "data_science",
    "project_manager": "carlos_himura",
    "project_name": "nba_binary_classification",
    "mlflow.note.content": experiment_description,
}
experiment_name = "XGBoost_Classification_No_Feat_Eng"
xgboost_experiment = client.create_experiment(name = experiment_name, tags = experiment_tags)

search_experiment = client.search_experiments(
  filter_string="tags.`project_name` = 'nba_binary_classification'"
)
print("Searching by Experiment Tag - Project Name:")
print(search_experiment[0])

Searching by Experiment Tag - Project Name:
<Experiment: artifact_location='mlflow-artifacts:/463662006620502774', creation_time=1764682857567, experiment_id='463662006620502774', last_update_time=1764682857567, lifecycle_stage='active', name='XGBoost_Classification_No_Feat_Eng', tags={'mlflow.note.content': 'This is the WZ challenge about NBA player '
                        'performanceThis experiment contains an XGBoost model '
                        'with hyperparameter tuning using Hyperopt.',
 'project_manager': 'carlos_himura',
 'project_name': 'nba_binary_classification',
 'team': 'data_science'}>


Logging the Initial MLflow Run inside the Created Experiment

In [18]:
# --- Logging the initial model to MLflow ---
import mlflow
import mlflow.xgboost
from mlflow.models import infer_signature

# Use the fluent API to set the tracking uri and the active experiment
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Sets the current active experiment to the "XGBoost_Classification_No_Feat_Eng" experiment and returns the Experiment metadata
xgboost_experiment = mlflow.set_experiment("XGBoost_Classification_No_Feat_Eng")

# Define a run name for this iteration of training. If not set, a unique name will be auto-generated 
run_name = "XGB_No_Feat_run_v5"

# Initiate the MLflow run context
with mlflow.start_run(run_name=run_name) as run:

 # Log run tags
  mlflow.set_tag("model_type", "xgboost")
  mlflow.set_tag("feature_engineering", "none")
  mlflow.set_tag("hyperopt", "none")
  mlflow.set_tag("class_weighting", "yes")
  mlflow.set_tag("author", "carlos_himura")

 # Log the parameters used for the model fit
  mlflow.log_params(initial_clf_params)

  # Log the metrics that were calculated during validation
  
  # Validation metrics
  mlflow.log_metric("val_accuracy", float(val_acc))
  mlflow.log_metric("val_roc_auc", float(val_roc))
  mlflow.log_metric("val_precision", float(val_prec))
  mlflow.log_metric("val_recall", float(val_rec))
  mlflow.log_metric("val_f1_score", float(val_f1))

  # Test metrics
  mlflow.log_metric("test_accuracy", float(test_acc))
  mlflow.log_metric("test_roc_auc", float(test_roc))
  mlflow.log_metric("test_precision", float(test_prec))
  mlflow.log_metric("test_recall", float(test_rec))
  mlflow.log_metric("test_f1_score", float(test_f1))

 # Validation Confusion Matrix as CSV
  val_cm_df = pd.DataFrame(val_cm_1)
  val_cm_df = pd.DataFrame(val_cm_1, 
                         index=["Pred_Pos_1 (TP-FP)", "Pred_Neg_0 (FN-TN)"], 
                         columns=["Act_Pos_1", "Act_Neg_0"])
  val_cm_df.to_csv("validation_confusion_matrix.csv")
  mlflow.log_artifact("validation_confusion_matrix.csv")

  # Test Confusion Matrix as CSV
  test_cm_df = pd.DataFrame(test_cm_1)
  test_cm_df = pd.DataFrame(test_cm_1,
                          index=["Pred_Pos_1 (TP-FP)", "Pred_Neg_0 (FN-TN)"],
                          columns=["Act_Pos_1", "Act_Neg_0"])
  test_cm_df.to_csv("test_confusion_matrix.csv")
  mlflow.log_artifact("test_confusion_matrix.csv")

  # Generate predictions for signature inference
  y_pred = xgb_clf.predict(X_1_test)

  # Create model signature and input example
  signature = infer_signature(X_1_test, y_pred)
  input_example = X_1_test.iloc[:5]

  # Log an instance of the trained model for later use
  mlflow.xgboost.log_model(
        xgb_clf, 
        name="xgb_clf",
        registered_model_name="XGB_Class_No_Feat_Eng", # If the model name doesn’t exist MLflow creates it. If it already exists MLflow automatically creates a new version
        input_example=input_example, # A small, representative example of the kind of input the model expects
        signature=signature # Model signature describing model input and output schema
        )
  

print(f"✅ Run '{run_name}' completed successfully.")

c:\Users\carlo\OneDrive\Documents\python_virtual_env\data_science_1\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Registered model 'XGB_Class_No_Feat_Eng' already exists. Creating a new version of this model...
2025/12/02 14:50:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB_Class_No_Feat_Eng, version 4
Created version '4' of model 'XGB_Class_No_Feat_Eng'.


🏃 View run XGB_No_Feat_run_v5 at: http://127.0.0.1:5000/#/experiments/463662006620502774/runs/855beea0acf54ed6b945b70efdb5e53b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/463662006620502774
✅ Run 'XGB_No_Feat_run_v5' completed successfully.


Second Model Training with Hyperopt and without Feature Engineering

Why Hyperopt?
- Performs automated hyperparameter tuning using intelligent search algorithms like Bayesian Optimization / TPE.
- Instead of trying random values blindly, Bayesian Optimization learns from previous trials and chooses better hyperparameters over time.

How is it work?
1. Search space = ingredients + allowed amounts --- Tell Hyperopt what values it can try
2. Objective function = taste test score --- Hyperopt tests a parameter set and gets a score
3. TPE algorithm = learns which combinations taste good --- Hyperopt runs Bayesian optimization (TPE) to find the best params
4. Final model = your final perfect dish --- Use best params on full dataset

In [ ]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# The ratio for scale_pos_weight is calculated from your class counts (3212 / 2188)
CLASS_WEIGHT_RATIO = 3212 / 2188 # 0 Class / 1 Class

# --- 1. Define the Search Space ---
# Defining the range of values that Hyperopt will test for each hyperparameter.
# Using the defaults as a guide to create the ranges.

search_space = {
    "n_estimators":     hp.quniform("n_estimators", 100, 600, 25), # Number of trees
    "max_depth":        hp.quniform("max_depth", 3, 10, 1), # Maximum depth of each tree
    "learning_rate":    hp.loguniform("learning_rate", np.log(0.01), np.log(0.3)), # Boosting learning rate
    "subsample":        hp.uniform("subsample", 0.6, 1.0), # % of rows sampled per tree
    "colsample_bytree": hp.uniform("colsample_bytree", 0.6, 1.0), # % of features sampled per tree
    "gamma":            hp.uniform("gamma", 0, 5), # Minimum loss reduction required to make a further partition on a leaf node of the tree
    "min_child_weight": hp.quniform("min_child_weight", 1, 10, 1), # It acts as a regularization term to prevent the model from learning relationships that are too specific to the training data
}

# --- 2. Define the objective function ---
def objective(params):

    # Convert integer params
    params["n_estimators"]     = int(params["n_estimators"])
    params["max_depth"]        = int(params["max_depth"])
    params["min_child_weight"] = int(params["min_child_weight"])

    model = XGBClassifier(
        objective="binary:logistic", # binary classification funtion loss
        eval_metric="logloss", # Evaluation metric
        random_state=12345, # Reproducibility
        tree_method="hist", # Fast + recommended
        scale_pos_weight=CLASS_WEIGHT_RATIO, # BALANCE THE CLASSES
        **params
    )

    model.fit(
        X_1_train, y_1_train,
        eval_set=[(X_1_valid, y_1_valid)],
        verbose=False
    )

    # Predict validation
    y_val_pred = model.predict(X_1_valid) # compute metrics that need discrete labels: accuracy, precision, recall, F1-score, confusion matrix, etc (Ratios)
    y_val_proba = model.predict_proba(X_1_valid)[:, 1] # probability of the positive class (used for ROC-AUC and any thresholding decisions)

    val_roc = roc_auc_score(y_1_valid, y_val_proba)  # Use ROC-AUC to optimize

    return {"loss": -val_roc, "status": STATUS_OK}

# --- 3. Run Hyperopt Search ---
trials = Trials()

best_params = fmin(
    fn=objective, # Function to minimize
    space=search_space,
    algo=tpe.suggest, # Search algorithm (TPE = Tree-structured Parzen Estimator)
    max_evals=30, # Number of different parameter sets to try
    trials=trials, # Object to store all results
    rstate=np.random.default_rng(42)
)

# Convert integer hyperparameters returned by Hyperopt
best_params["n_estimators"]     = int(best_params["n_estimators"])
best_params["max_depth"]        = int(best_params["max_depth"])
best_params["min_child_weight"] = int(best_params["min_child_weight"])


print("\n Best Hyperopt Parameters:")
print(best_params)

# --- 4. TRAIN FINAL MODEL WITH BEST PARAMETERS ---

xgb_clf = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=12345,
    tree_method="hist",
    **best_params
)

xgb_clf.fit(
    X_1_train, y_1_train,
    eval_set=[(X_1_valid, y_1_valid)],
    verbose=False
)


# VALIDATION PREDICTIONS
y_val_pred = xgb_clf.predict(X_1_valid) # compute metrics that need discrete labels: accuracy, precision, recall, F1-score, confusion matrix, etc (Ratios)
y_val_proba = xgb_clf.predict_proba(X_1_valid)[:, 1] # probability of the positive class (used for ROC-AUC and any thresholding decisions)

val_acc_2  = accuracy_score(y_1_valid, y_val_pred) # Fraction of correct predictions
val_roc_2  = roc_auc_score(y_1_valid, y_val_proba) # Measures the model’s ability to distinguish between classes 
val_prec_2 = precision_score(y_1_valid, y_val_pred) # Fraction of true positives among predicted positives
val_rec_2  = recall_score(y_1_valid, y_val_pred) # Fraction of true positives among actual positives
val_f1_2   = f1_score(y_1_valid, y_val_pred)  # Harmonic mean of precision and recall

print("\n📊 Validation Results:")
print(f"Accuracy: {val_acc_2:.4f}")
print(f"ROC AUC: {val_roc_2:.4f}")
print(f"Precision: {val_prec_2:.4f}")
print(f"Recall (Sensitivity): {val_rec_2:.4f}")
print(f"F1-Score: {val_f1_2:.4f}")

# Confusion Matrix (Validation)
val_cm_2 = confusion_matrix(y_1_valid, y_val_pred)
print("\n🔷 Validation Confusion Matrix:")
print("[[True_Pos False_Pos]")
print("[False_Neg True_Neg]]")
print(val_cm_2)

# TEST PREDICTIONS
y_test_pred = xgb_clf.predict(X_1_test) # compute metrics that need discrete labels: accuracy, precision, recall, F1-score, confusion matrix, etc (Ratios)
y_test_proba = xgb_clf.predict_proba(X_1_test)[:, 1] # probability of the positive class (used for ROC-AUC and any thresholding decisions)

test_acc_2  = accuracy_score(y_1_test, y_test_pred) # Fraction of correct predictions
test_roc_2  = roc_auc_score(y_1_test, y_test_proba) # Measures the model’s ability to distinguish between classes
test_prec_2 = precision_score(y_1_test, y_test_pred) # Fraction of true positives among predicted positives
test_rec_2  = recall_score(y_1_test, y_test_pred) # Fraction of true positives among actual positives
test_f1_2   = f1_score(y_1_test, y_test_pred) # Harmonic mean of precision and recall

print("\n----------------------------------------")
print("📊 Test Results:")
print(f"Accuracy: {test_acc_2:.4f}")
print(f"ROC AUC: {test_roc_2:.4f}")
print(f"Precision: {test_prec_2:.4f}")
print(f"Recall (Sensitivity): {test_rec_2:.4f}")
print(f"F1-Score: {test_f1_2:.4f}")

# Confusion Matrix (Test)
test_cm_2 = confusion_matrix(y_1_test, y_test_pred)
print("\n🔷 Test Confusion Matrix:")
print("[[True_Pos False_Pos]")
print("[False_Neg True_Neg]]")
print(test_cm_2)

  0%|          | 0/30 [00:00<?, ?trial/s, best loss=?]

c:\Users\carlo\OneDrive\Documents\python_virtual_env\data_science_1\Lib\site-packages\hyperopt\atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


100%|██████████| 30/30 [00:27<00:00,  1.10trial/s, best loss: -0.9310568905224416]

 Best Hyperopt Parameters:
{'colsample_bytree': np.float64(0.9806881870543972), 'gamma': np.float64(3.94634986154594), 'learning_rate': np.float64(0.045974998070504425), 'max_depth': 3, 'min_child_weight': 10, 'n_estimators': 250, 'subsample': np.float64(0.8320612810134949)}

📊 Validation Results:
Accuracy: 0.8528
ROC AUC: 0.9315
Precision: 0.8225
Recall (Sensitivity): 0.8077
F1-Score: 0.8151

🔷 Validation Confusion Matrix:
[[True_Pos False_Pos]
[False_Neg True_Neg]]
[[951 126]
 [139 584]]

----------------------------------------
📊 Test Results:
Accuracy: 0.8478
ROC AUC: 0.9302
Precision: 0.8092
Recall (Sensitivity): 0.7881
F1-Score: 0.7985

🔷 Test Confusion Matrix:
[[True_Pos False_Pos]
[False_Neg True_Neg]]
[[983 128]
 [146 543]]


Logging the Differents MLflow Runs inside the Created Experiment After Hyperopt

In [22]:
# --- Logging the initial model to MLflow ---
import mlflow
import mlflow.xgboost
from mlflow.models import infer_signature
import pandas as pd

# Use the fluent API to set the tracking uri and the active experiment
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Sets the current active experiment to the "XGBoost_Classification_No_Feat_Eng" experiment and returns the Experiment metadata
xgboost_experiment = mlflow.set_experiment("XGBoost_Classification_No_Feat_Eng")

# Define a run name for this iteration of training. If not set, a unique name will be auto-generated 
run_name = "XGB_No_Feat_run_v6"

# Initiate the MLflow run context
with mlflow.start_run(run_name=run_name) as run:

 # Log run tags
  mlflow.set_tag("model_type", "xgboost")
  mlflow.set_tag("feature_engineering", "none")
  mlflow.set_tag("hyperopt", "yes")
  mlflow.set_tag("class_weighting", "yes")
  mlflow.set_tag("author", "carlos_himura")

 # FIX: Log the scale_pos_weight parameter explicitly
  mlflow.log_param("scale_pos_weight", CLASS_WEIGHT_RATIO) 

 # Log the parameters used for the model fit
  mlflow.log_params(best_params)

  # Log the metrics that were calculated during validation
  
  # Validation metrics
  mlflow.log_metric("val_accuracy", float(val_acc_2))
  mlflow.log_metric("val_roc_auc", float(val_roc_2))
  mlflow.log_metric("val_precision", float(val_prec_2))
  mlflow.log_metric("val_recall", float(val_rec_2))
  mlflow.log_metric("val_f1_score", float(val_f1_2))

  # Test metrics
  mlflow.log_metric("test_accuracy", float(test_acc_2))
  mlflow.log_metric("test_roc_auc", float(test_roc_2))
  mlflow.log_metric("test_precision", float(test_prec_2))
  mlflow.log_metric("test_recall", float(test_rec_2))
  mlflow.log_metric("test_f1_score", float(test_f1_2))

  # Validation Confusion Matrix as CSV
  val_cm_df = pd.DataFrame(val_cm_2)
  val_cm_df = pd.DataFrame(val_cm_2, 
                         index=["Pred_Pos_1 (TP-FP)", "Pred_Neg_0 (FN-TN)"], 
                         columns=["Act_Pos_1", "Act_Neg_0"])
  val_cm_df.to_csv("validation_confusion_matrix.csv")
  mlflow.log_artifact("validation_confusion_matrix.csv")

  # Test Confusion Matrix as CSV
  test_cm_df = pd.DataFrame(test_cm_2)
  test_cm_df = pd.DataFrame(test_cm_2,
                          index=["Pred_Pos_1 (TP-FP)", "Pred_Neg_0 (FN-TN)"],
                          columns=["Act_Pos_1", "Act_Neg_0"])
  test_cm_df.to_csv("test_confusion_matrix.csv")
  mlflow.log_artifact("test_confusion_matrix.csv")

  # Generate predictions for signature inference
  y_pred = xgb_clf.predict(X_1_test)

  # Create model signature and input example
  signature = infer_signature(X_1_test, y_pred)
  input_example = X_1_test.iloc[:5]

  # Log an instance of the trained model for later use
  mlflow.xgboost.log_model(
        xgb_clf, 
        name="xgb_clf",
        registered_model_name="XGB_Class_No_Feat_Eng", # If the model name doesn’t exist MLflow creates it. If it already exists MLflow automatically creates a new version
        input_example=input_example, # A small, representative example of the kind of input the model expects
        signature=signature # Model signature describing model input and output schema
        )
  

print(f"✅ Run '{run_name}' completed successfully.")

c:\Users\carlo\OneDrive\Documents\python_virtual_env\data_science_1\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Registered model 'XGB_Class_No_Feat_Eng' already exists. Creating a new version of this model...
2025/12/02 15:11:48 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB_Class_No_Feat_Eng, version 5


🏃 View run XGB_No_Feat_run_v6 at: http://127.0.0.1:5000/#/experiments/463662006620502774/runs/7f161a3cd2bc463caa09d8ddb42af2df
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/463662006620502774
✅ Run 'XGB_No_Feat_run_v6' completed successfully.


Created version '5' of model 'XGB_Class_No_Feat_Eng'.


Metrics description 
- ROC-AUC: Measures the model’s ability to distinguish between classes
- Accuracy: Overall correctness of the model.
- Precision: Out of all predicted positives, how many were correct?
- Recall (Sensitivity): Out of all actual positives, how many were detected?
- F1 Score: Balance between precision and recall.

Metrics and Model Conclusion:

NBA Classification — What Metrics Should We Use to Choose the Best Model?
- The goal is to predict whether an NBA player would score over a certain threshold (binary classification).
- The 60/40 imbalance has been handled with class_weight.
- It should focus on metrics that are sensitive to both False Positives and False Negatives. Keeping in mind an accuracy of 65% might just mean it predicted the majority class correctly.
- Primary the ROC AUC: It evaluates the model's ability to distinguish between the two classes across all possible classification thresholds.
- Secondary F1-Score: is the harmonic mean of Precision and Recall. It provides a single number that requires the model to have high scores on both.
- Thrid Precision and Recall: Minimizing False Positives / Minimizing False Negatives.

Challenge especification: The dataset includes natural variability. Well-built models
typically achieve around 0.80 accuracy and 0.85–0.90 ROC-AUC without overfitting.


Selecting the Best Model, Parameters, and Metrics from MLflow

In [29]:
from mlflow import MlflowClient

client = MlflowClient()
experiment = client.get_experiment_by_name("XGBoost_Classification_No_Feat_Eng")

# Fetch all runs for the experiment
runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["metrics.test_roc_auc DESC"],  # Sort by best test ROC-AUC and Accuracy
    #order_by=["metrics.test_roc_auc DESC", "metrics.test_accuracy DESC"],  # Sort by best test ROC-AUC and Accuracy
)

# Select the best run (first in the sorted list)
best_run = runs[0]
print(f"Best Run Name: {best_run.info.run_name}")
print(f"Best Run ID: {best_run.info.run_id}")

summary_data = {
    "Test Accuracy": [best_run.data.metrics.get("test_accuracy")],
    "Test ROC-AUC": [best_run.data.metrics.get("test_roc_auc")],
    "Test Precision": [best_run.data.metrics.get("test_precision")],
    "Test Recall": [best_run.data.metrics.get("test_recall")],
    "Test F1-Score": [best_run.data.metrics.get("test_f1_score")],
}

# Create a pandas DataFrame for a clean table
summary_df = pd.DataFrame(summary_data)

# Display neatly
print("\nBest Run Summary Metrics:")
display(summary_df.style.hide(axis="index"))

# Extract best run parameters
params_data = best_run.data.params

# Convert to a DataFrame
params_df = pd.DataFrame(list(params_data.items()), columns=["Parameter", "Value"])

# Display nicely without index
print("Best Run Parameters:")
display(params_df.style.hide(axis="index"))


Best Run Name: XGB_No_Feat_run_v6
Best Run ID: 7f161a3cd2bc463caa09d8ddb42af2df

Best Run Summary Metrics:


Test Accuracy,Test ROC-AUC,Test Precision,Test Recall,Test F1-Score
0.847778,0.930173,0.809240,0.788099,0.798529


Best Run Parameters:


Parameter,Value
colsample_bytree,0.9806881870543972
gamma,3.94634986154594
learning_rate,0.045974998070504425
max_depth,3
min_child_weight,10
n_estimators,250
scale_pos_weight,1.4680073126142597
subsample,0.8320612810134949
